In [14]:
# package and data importing

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols 
from linearmodels.panel import PanelOLS

wri_df = pd.read_excel('PS_WRI_data_nonlag_TRIMMED.xlsx')

In [22]:
# building our data for panel format application -- WRI specific

# --- Data Preparation ---

# 1. Melt to long format
country_col = [col for col in wri_df.columns if 'country' in col.lower()][0] #Find the country column, case insensitive.


long_data = pd.melt(wri_df,
                    id_vars=[country_col],
                    var_name='year_category',
                    value_name='score')

# 2. Extract year and category - SIMPLIFIED (Correct for consistent format)
long_data[['year', 'category']] = long_data['year_category'].str.split('_', expand=True)
long_data['year'] = long_data['year'].astype(int)

long_data.drop(columns=['year_category'], inplace=True)
long_data.rename(columns={country_col: 'country'}, inplace=True)

# --- Panel Regression ---
long_data = long_data.set_index(['country', 'year'])
regressors = pd.get_dummies(long_data['category'], drop_first=True)
model_fe_panel = PanelOLS(long_data['score'], regressors, entity_effects=True, time_effects=True).fit()

print("Panel Regression Results:")
print(model_fe_panel)
print("\nRobust standard errors:")
print(model_fe_panel.std_errors)

# --- Exporting Results to a Table ---

# 1. Create a dictionary to store the results
results_dict = {
    'Coefficient': model_fe_panel.params,
    'Std. Error': model_fe_panel.std_errors,
    't-statistic': model_fe_panel.tstats,
    'p-value': model_fe_panel.pvalues
}

# 2. Convert the dictionary to a Pandas DataFrame
results_df = pd.DataFrame(results_dict)

# 3. Format the table (optional)
results_df = results_df.round(3)  # Round to 3 decimal places
results_df.index.name = "Variable"  # Set index name

# 4. Export to CSV or other formats
results_df.to_csv('panel_regression_results.csv')  # Export to CSV
# Or export to Excel:
# results_df.to_excel('panel_regression_results.xlsx')
# Or display in a formatted way in your notebook:
print(results_df)

# --- Additional Information (Optional) ---
# You might want to include additional information in your table:

# R-squared
r_squared = model_fe_panel.rsquared
print(f"R-squared: {r_squared:.3f}")

# Number of observations
nobs = model_fe_panel.nobs
print(f"Number of Observations: {nobs}")

# Entity and Time Effects included? - CORRECTED
def effects_status(model_result, effect_type):
    """Checks and returns the status of entity or time effects."""
    try:
        if effect_type == "entity":
            included = model_result.params.index.str.startswith("entity").any()
        elif effect_type == "time":
            included = model_result.params.index.str.startswith("time").any()
        else:
            return "Unknown effect type"
        return "Included" if included else "Not Included"
    except AttributeError: #For models without effects, like PooledOLS
      return "Not Applicable"

entity_effects_status = effects_status(model_fe_panel, "entity")
time_effects_status = effects_status(model_fe_panel, "time")


# Entity and Time Effects included?
print(f"Entity Effects: {entity_effects_status}")
print(f"Time Effects: {time_effects_status}")

Panel Regression Results:
                          PanelOLS Estimation Summary                           
Dep. Variable:                  score   R-squared:                        0.7637
Estimator:                   PanelOLS   R-squared (Between):             -1.8681
No. Observations:                3000   R-squared (Within):               0.7565
Date:                Mon, Feb 17 2025   R-squared (Overall):             -1.0394
Time:                        22:25:29   Log-likelihood                -1.179e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1907.5
Entities:                          20   P-value                           0.0000
Avg Obs:                       150.00   Distribution:                  F(5,2951)
Min Obs:                       150.00                                           
Max Obs:                       150.00   F-statistic (robust):             1907.5
  

In [27]:
# Cross-sectional regression approach
import statsmodels.formula.api as smf 
year_to_analyze = 2022
cross_copy = long_data.reset_index()

# filtering to target year:
cross_sectional_data = cross_copy[cross_copy['year'] == year_to_analyze].copy()

# aggregating our multiple observations per country
cross_sectional_data = cross_sectional_data.groupby('country')['score'].sum().reset_index()

# --- Cross-Sectional Regression ---
csr_model = smf.ols("score ~ 1", data=cross_sectional_data).fit() #The 1 represents the intercept.

print(csr_model.summary())


                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 18 Feb 2025   Prob (F-statistic):                nan
Time:                        08:42:30   Log-Likelihood:                -109.57
No. Observations:                  20   AIC:                             221.1
Df Residuals:                      19   BIC:                             222.1
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    238.0225     13.292     17.907      0.0